### Orginal Notebook Created by CIEP / Global DDM COE
#### Nidhi Sawhney, Stojan Maleschlijski & Ian Henry

In [1]:


import tensorflow as tf
from keras import backend as K
from keras.models import load_model, Sequential, Model
from keras.layers import Embedding, InputLayer, Convolution1D, MaxPooling1D, SpatialDropout1D
from keras.layers.core import Flatten,Dense,Dropout

sess = tf.Session()
K.set_session(sess)
K.set_learning_phase(0)  # all new operations will be in test mode from now on

/Users/i049374/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# preprocessing function that expects a string of words as Ints seperated by spaces
def preprocess (txt_input):
    sparse_tokenized_input = tf.string_split(txt_input,delimiter=' ')
    tokenized_input = tf.sparse_tensor_to_dense(sparse_tokenized_input, default_value='0')
    token_idxs = tf.string_to_number(tokenized_input, out_type=tf.float32)
    inputlength = tf.size(token_idxs)
    # Max Number of Words in Sentance 40
    padding = 40 - inputlength
    token_idxs_padded = tf.pad(token_idxs, [[0,0],[padding,0]])
    token_idxs_embedding = tf.slice(token_idxs_padded, [0,0], [1,40])
    return token_idxs_embedding;

In [3]:
# Reload the Keras Model
model = load_model('./Models/FakeNews-v7.h5')

txt_input = tf.placeholder(tf.string, name='txt_input')
token_idxs_embedding = preprocess(txt_input)

# Recreate Binary Classification Model
text_fn_model = Sequential([
    InputLayer(input_tensor=token_idxs_embedding,input_shape=(1,40)),
    Embedding(5000, 32, input_length=40),
    SpatialDropout1D(0.2),
    Dropout(0.25),
    Convolution1D(64, 5, padding='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid',name='prediction')])

text_fn_model.name='fakenews'
config = model.get_config()
weights = model.get_weights()
text_fn_model.set_weights(weights)

In [4]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

# You must increment the number below if you run this.  This is the Model version for Serving
export_path = './Models/FakeNews-Serving/1'
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'text': txt_input},
                               outputs={'labels': text_fn_model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                        signature_def_map={signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'./Models/FakeNews-Serving/1/saved_model.pb'
